In order to create the annual estimate of wildfire smoke in Salina, Kansas, I am going to read the USGS_Wildland_Fire_Combined_Dataset.json which has been downloaded from the [USGS Website from the Combined wildland fire datasets for the United States and certain territories, 1800s-Present (combined wildland fire polygons)](https://www.sciencebase.gov/catalog/item/61aa537dd34eb622f699df81) dataset. 

In order to explore the dataset, I will be reading the JSON data with the help of [this](https://drive.google.com/file/d/1TwCkvdaw0MxJzW7NSDg6XxYQ0dvaS44I/view) solution provided by Professor McDonald. I will also be using code from the [Wildfire Proximity Computational Example notebook provided](https://colab.research.google.com/drive/1qNI6hji8CvDeBsnLDAhJXvaqf2gcg8UV#scrollTo=DwLfwdk9rXcd). The notebook is licensed for use in DATA 512, a course in the UW MS Data Science degree program. This code is provided under the Creative Commons CC-BY license. Revision 1.0 - August 13, 2023

# Section 1: Importing Libraries and Initial Setup

I'm starting by bringing in some essential tools to help with our data work. We've got Pandas for handling data, geospatial libraries for calculating distances, and a custom module we obtained from our course material to deal with wildfire data.

In [2]:
import pandas as pd
import json, os, time
from geopy.distance import geodesic
import sys, json
from wildfire.Reader import Reader as WFReader
from pyproj import Transformer, Geod
import geojson

- You might need to install pyproj and geojson, you can do so via pip.
- The 'wildfire' module has been taken from the course drive and can be found [here](https://drive.google.com/file/d/1TwCkvdaw0MxJzW7NSDg6XxYQ0dvaS44I/view).
- This module includes a Reader object for reading GeoJSON files linked to the wildfire dataset.
- It also provides a sample datafile called Wildfire_short_sample.json that contains California wildfires data.

# Section 2: Coordinate Conversion Function

We firstly take the geometry of a fire feature, extract the largest ring (i.e., the largest boundary of the fire) and convert all of the points in that ring from the ESRI:102008 coordinate system to EPSG:4326 coordinates. This is going to be useful for our geospatial work.

In [3]:
def convert_ring_to_epsg4326(ring_data=None):
    converted_ring = list()
    
    to_epsg4326 = Transformer.from_crs("ESRI:102008","EPSG:4326")
    for coord in ring_data:
        lat,lon = to_epsg4326.transform(coord[0],coord[1])
        new_coord = lat,lon
        converted_ring.append(new_coord)
    return converted_ring

# Section 3: Finding Shortest Distance to Fire Perimeter

Our next is step is finding shortest distance between Salina and each wildfire. The following code finds the point on the perimiter with the shortest distance to the city (place) and returns the distance as well as the lat,lon of the perimeter point.   

In [4]:
def shortest_distance_from_place_to_fire_perimeter(place=None,ring_data=None):
    ring = convert_ring_to_epsg4326(ring_data)    
    geodcalc = Geod(ellps='WGS84')
    closest_point = list()
    for point in ring:
        d = geodcalc.inv(place[1],place[0],point[1],point[0])
        distance_in_miles = d[2]*0.00062137
        if not closest_point:
            closest_point.append(distance_in_miles)
            closest_point.append(point)
        elif closest_point and closest_point[0]>distance_in_miles:
            closest_point = list()
            closest_point.append(distance_in_miles)
            closest_point.append(point)
    return closest_point

I am going to read the GeoJSON data, USGS_Wildland_Fire_Combined_Dataset.json, which has been downloaded from the [USGS Website from the Combined wildland fire datasets for the United States and certain territories, 1800s-Present (combined wildland fire polygons)](https://www.sciencebase.gov/catalog/item/61aa537dd34eb622f699df81) dataset. 

For reference, SAMPLE_DATA_FILENAME is the path to USGS_Wildland_Fire_Combined_Dataset.json 


# Section 4: Loading GeoJSON Data

In [5]:
SAMPLE_DATA_FILENAME = '/Users/aviva/Desktop/MSDS/quarter_4/Human Centered Data Science/Part 1/GeoJSON Exports/USGS_Wildland_Fire_Combined_Dataset.json'

Pulling metadata to see what we're working with!

In [6]:
print(f"Attempting to open '{SAMPLE_DATA_FILENAME}' with wildfire.Reader() object")
wfreader = WFReader(SAMPLE_DATA_FILENAME)
print()

header_dict = wfreader.header()
header_keys = list(header_dict.keys())
#print("The header has the following keys:")
#print(gj_keys)
print()
print("Header Dictionary")
print(json.dumps(header_dict,indent=4))

Attempting to open '/Users/aviva/Desktop/MSDS/quarter_4/Human Centered Data Science/Part 1/GeoJSON Exports/USGS_Wildland_Fire_Combined_Dataset.json' with wildfire.Reader() object


Header Dictionary
{
    "displayFieldName": "",
    "fieldAliases": {
        "OBJECTID": "OBJECTID",
        "USGS_Assigned_ID": "USGS Assigned ID",
        "Assigned_Fire_Type": "Assigned Fire Type",
        "Fire_Year": "Fire Year",
        "Fire_Polygon_Tier": "Fire Polygon Tier",
        "Fire_Attribute_Tiers": "Fire Attribute Tiers",
        "GIS_Acres": "GIS_Acres",
        "GIS_Hectares": "GIS_Hectares",
        "Source_Datasets": "Source Datasets",
        "Listed_Fire_Types": "Listed Fire Types",
        "Listed_Fire_Names": "Listed Fire Names",
        "Listed_Fire_Codes": "Listed Fire Codes",
        "Listed_Fire_IDs": "Listed Fire IDs",
        "Listed_Fire_IRWIN_IDs": "Listed Fire IRWIN IDs",
        "Listed_Fire_Dates": "Listed Fire Dates",
        "Listed_Fire_Causes": "Listed Fire Causes",
 

# Section 5: Loading and Processing Fire Features

Here, I'm looping through all the wildfire features in our dataset and storing them in a list. We're also keeping track of how many we've processed! 

In [7]:
MAX_FEATURE_LOAD = 100
feature_list = list()
feature_count = 0

wfreader.rewind()
feature = wfreader.next()
while feature:
    feature_list.append(feature)
    feature_count += 1
    if (feature_count % 100) == 0:
        print(f"Loaded {feature_count} features")
    '''if feature_count >= MAX_FEATURE_LOAD:
        break'''
    feature = wfreader.next()
print(f"Loaded a total of {feature_count} features")

print(f"Variable 'feature_list' contains {len(feature_list)} features")

Loaded 100 features
Loaded 200 features
Loaded 300 features
Loaded 400 features
Loaded 500 features
Loaded 600 features
Loaded 700 features
Loaded 800 features
Loaded 900 features
Loaded 1000 features
Loaded 1100 features
Loaded 1200 features
Loaded 1300 features
Loaded 1400 features
Loaded 1500 features
Loaded 1600 features
Loaded 1700 features
Loaded 1800 features
Loaded 1900 features
Loaded 2000 features
Loaded 2100 features
Loaded 2200 features
Loaded 2300 features
Loaded 2400 features
Loaded 2500 features
Loaded 2600 features
Loaded 2700 features
Loaded 2800 features
Loaded 2900 features
Loaded 3000 features
Loaded 3100 features
Loaded 3200 features
Loaded 3300 features
Loaded 3400 features
Loaded 3500 features
Loaded 3600 features
Loaded 3700 features
Loaded 3800 features
Loaded 3900 features
Loaded 4000 features
Loaded 4100 features
Loaded 4200 features
Loaded 4300 features
Loaded 4400 features
Loaded 4500 features
Loaded 4600 features
Loaded 4700 features
Loaded 4800 features
L

Loaded 38100 features
Loaded 38200 features
Loaded 38300 features
Loaded 38400 features
Loaded 38500 features
Loaded 38600 features
Loaded 38700 features
Loaded 38800 features
Loaded 38900 features
Loaded 39000 features
Loaded 39100 features
Loaded 39200 features
Loaded 39300 features
Loaded 39400 features
Loaded 39500 features
Loaded 39600 features
Loaded 39700 features
Loaded 39800 features
Loaded 39900 features
Loaded 40000 features
Loaded 40100 features
Loaded 40200 features
Loaded 40300 features
Loaded 40400 features
Loaded 40500 features
Loaded 40600 features
Loaded 40700 features
Loaded 40800 features
Loaded 40900 features
Loaded 41000 features
Loaded 41100 features
Loaded 41200 features
Loaded 41300 features
Loaded 41400 features
Loaded 41500 features
Loaded 41600 features
Loaded 41700 features
Loaded 41800 features
Loaded 41900 features
Loaded 42000 features
Loaded 42100 features
Loaded 42200 features
Loaded 42300 features
Loaded 42400 features
Loaded 42500 features
Loaded 426

Loaded 75600 features
Loaded 75700 features
Loaded 75800 features
Loaded 75900 features
Loaded 76000 features
Loaded 76100 features
Loaded 76200 features
Loaded 76300 features
Loaded 76400 features
Loaded 76500 features
Loaded 76600 features
Loaded 76700 features
Loaded 76800 features
Loaded 76900 features
Loaded 77000 features
Loaded 77100 features
Loaded 77200 features
Loaded 77300 features
Loaded 77400 features
Loaded 77500 features
Loaded 77600 features
Loaded 77700 features
Loaded 77800 features
Loaded 77900 features
Loaded 78000 features
Loaded 78100 features
Loaded 78200 features
Loaded 78300 features
Loaded 78400 features
Loaded 78500 features
Loaded 78600 features
Loaded 78700 features
Loaded 78800 features
Loaded 78900 features
Loaded 79000 features
Loaded 79100 features
Loaded 79200 features
Loaded 79300 features
Loaded 79400 features
Loaded 79500 features
Loaded 79600 features
Loaded 79700 features
Loaded 79800 features
Loaded 79900 features
Loaded 80000 features
Loaded 801

Loaded 112300 features
Loaded 112400 features
Loaded 112500 features
Loaded 112600 features
Loaded 112700 features
Loaded 112800 features
Loaded 112900 features
Loaded 113000 features
Loaded 113100 features
Loaded 113200 features
Loaded 113300 features
Loaded 113400 features
Loaded 113500 features
Loaded 113600 features
Loaded 113700 features
Loaded 113800 features
Loaded 113900 features
Loaded 114000 features
Loaded 114100 features
Loaded 114200 features
Loaded 114300 features
Loaded 114400 features
Loaded 114500 features
Loaded 114600 features
Loaded 114700 features
Loaded 114800 features
Loaded 114900 features
Loaded 115000 features
Loaded 115100 features
Loaded 115200 features
Loaded 115300 features
Loaded 115400 features
Loaded 115500 features
Loaded 115600 features
Loaded 115700 features
Loaded 115800 features
Loaded 115900 features
Loaded 116000 features
Loaded 116100 features
Loaded 116200 features
Loaded 116300 features
Loaded 116400 features
Loaded 116500 features
Loaded 1166

# Section 6: Defining City Data

Specifying the coordinates for the city of Salina, Kansas

In [8]:
CITY_DATA = {
    'salina' :     {'city'   : 'salina', 
                    'latlon' : [38.8403, -97.6114]}}

# Section 7: Calculating Distances to Fire Perimeter

Calculating the distance from the closest edge of the fire to the town.

In [9]:
city_data = CITY_DATA["salina"]
fire_ids = []
shortest_dist_from_edge_list = []
features_processed = 0

There are some fires which aren't in ring shape. These are being processed in the try/except block to catch and deal with them.

In [10]:
for wf_feature in feature_list:
    try:
        ring_data = wf_feature['geometry']['rings'][0]
        distance = shortest_distance_from_place_to_fire_perimeter(city_data['latlon'], ring_data)
        fire_ids.append(wf_feature['attributes']['OBJECTID'])
        shortest_dist_from_edge_list.append(round(distance[0], 2))
    except KeyError:
        print("{0} fire has an irregular shape; skipping.".format(wf_feature['attributes']['OBJECTID']))

    features_processed += 1
    if features_processed % 1000 == 0:
        print("Processed {0} features".format(features_processed))
    if features_processed % 10000 == 0:
        dist_df = pd.DataFrame({'OBJECTID': fire_ids, 'shortest_dist': shortest_dist_from_edge_list})
        dist_df.to_csv('/Users/aviva/Desktop/MSDS/quarter_4/Human Centered Data Science/Part 1/GeoJSON Exports/intermediate_data/distance.csv', index=False)

distance_df = pd.DataFrame({'OBJECTID': fire_ids, 'shortest_dist': shortest_dist_from_edge_list})
distance_df.to_csv('/Users/aviva/Desktop/MSDS/quarter_4/Human Centered Data Science/Part 1/GeoJSON Exports/intermediate_data/distance.csv', index=False)

Processed 1000 features
Processed 2000 features
Processed 3000 features
Processed 4000 features
Processed 5000 features
Processed 6000 features
Processed 7000 features
Processed 8000 features
Processed 9000 features
Processed 10000 features
Processed 11000 features
Processed 12000 features
Processed 13000 features
Processed 14000 features
Processed 15000 features
Processed 16000 features
Processed 17000 features
Processed 18000 features
Processed 19000 features
Processed 20000 features
Processed 21000 features
Processed 22000 features
Processed 23000 features
Processed 24000 features
Processed 25000 features
Processed 26000 features
Processed 27000 features
Processed 28000 features
Processed 29000 features
Processed 30000 features
Processed 31000 features
Processed 32000 features
Processed 33000 features
Processed 34000 features
Processed 35000 features
Processed 36000 features
Processed 37000 features
Processed 38000 features
Processed 39000 features
Processed 40000 features
Processed

Reading the distances df

In [11]:
distance_df = pd.read_csv('/Users/aviva/Desktop/MSDS/quarter_4/Human Centered Data Science/Part 1/GeoJSON Exports/intermediate_data/distance.csv')

We will now use the following conditions to further filter the data. Your smoke estimate should adhere to the following conditions:  
- The estimate only considers the last 60 years of wildland fires (1963-2023).
- The estimate only considers fires that are within 1250 miles of your assigned city.
- An annual fire season will run from May 1st through October 31st.


First we keep only fires which occurr within 1250 miles from Salina.

In [12]:
miles_df = distance_df.loc[distance_df['shortest_dist']< 1250]
print("{0} fires occured within 1250 miles of Salina".format(len(miles_df)))

miles_df.to_csv('/Users/aviva/Desktop/MSDS/quarter_4/Human Centered Data Science/Part 1/GeoJSON Exports/intermediate_data/miles.csv', index = False)

104079 fires occured within 1250 miles of Salina


Now we further filter this by keeping only fires which occurred in or after 1963. Since the data we have is avaliable till 2020 we don't need to specify that.

In [13]:
wf_file = open('/Users/aviva/Desktop/MSDS/quarter_4/Human Centered Data Science/Part 1/GeoJSON Exports/USGS_Wildland_Fire_Combined_Dataset.json')
wf_dict = json.load(wf_file)

# Section 8: Filter and Extract Fire Data for years 1963 and after

We now filter and extract specific fire data entries with a 'Fire_Year' attribute greater than or equal to 1963 and stores relevant information in separate lists.

In [14]:
obj_id = []
fire_types = []
fire_years = []
acres = []
overlap_flags = []
fire_count = 0

for fire in wf_dict['features']:
    if fire['attributes']['Fire_Year'] >= 1963:
        obj_id.append(fire['attributes']['OBJECTID'])
        fire_types.append(fire['attributes']['Assigned_Fire_Type'])
        fire_years.append(fire['attributes']['Fire_Year'])
        acres.append(fire['attributes']['GIS_Acres'])
        overlap_flags.append(fire['attributes']['Overlap_Within_1_or_2_Flag'])
        fire_count += 1

        if fire_count % 1000 == 0:
            print("Processed {0} fires".format(fire_count))

Processed 1000 fires
Processed 2000 fires
Processed 3000 fires
Processed 4000 fires
Processed 5000 fires
Processed 6000 fires
Processed 7000 fires
Processed 8000 fires
Processed 9000 fires
Processed 10000 fires
Processed 11000 fires
Processed 12000 fires
Processed 13000 fires
Processed 14000 fires
Processed 15000 fires
Processed 16000 fires
Processed 17000 fires
Processed 18000 fires
Processed 19000 fires
Processed 20000 fires
Processed 21000 fires
Processed 22000 fires
Processed 23000 fires
Processed 24000 fires
Processed 25000 fires
Processed 26000 fires
Processed 27000 fires
Processed 28000 fires
Processed 29000 fires
Processed 30000 fires
Processed 31000 fires
Processed 32000 fires
Processed 33000 fires
Processed 34000 fires
Processed 35000 fires
Processed 36000 fires
Processed 37000 fires
Processed 38000 fires
Processed 39000 fires
Processed 40000 fires
Processed 41000 fires
Processed 42000 fires
Processed 43000 fires
Processed 44000 fires
Processed 45000 fires
Processed 46000 fir

In [15]:
feature_df = pd.DataFrame({'OBJECTID':obj_id,
                            'FireType':fire_types,
                            'FireYear':fire_years,
                            'GISAcres':acres,
                            'OverlapFlags' : overlap_flags})
feature_df.to_csv('/Users/aviva/Desktop/MSDS/quarter_4/Human Centered Data Science/Part 1/GeoJSON Exports/intermediate_data/features.csv', index=False)

In [16]:
feature_df 

,OBJECTID,FireType,FireYear,GISAcres,OverlapFlags
0,14299,Wildfire,1963,40992.458271,None
1,14300,Wildfire,1963,25757.090203,None
2,14301,Wildfire,1963,45527.210986,None
3,14302,Wildfire,1963,10395.010334,None
4,14303,Wildfire,1963,9983.605738,None
...,...,...,...,...,...
117573,135057,Prescribed Fire,2020,16.412148,"Caution, this Prescribed Fire in 2020 overlaps..."
117574,135058,Prescribed Fire,2020,7.050837,"Caution, this Prescribed Fire in 2020 overlaps..."
117575,135059,Prescribed Fire,2020,9.342668,"Caution, this Prescribed Fire in 2020 overlaps..."
117576,135060,Prescribed Fire,2020,0.996962,None


In [17]:
feature_df = pd.read_csv('/Users/aviva/Desktop/MSDS/quarter_4/Human Centered Data Science/Part 1/GeoJSON Exports/intermediate_data/features.csv')
miles_df= pd.read_csv('/Users/aviva/Desktop/MSDS/quarter_4/Human Centered Data Science/Part 1/GeoJSON Exports/intermediate_data/miles.csv')

Next we inner join the post-1963 fire attributes with their distances to calculate the smoke estimate. We will save this dataframe for later use in the data_processing script.

In [23]:
miles_df

,OBJECTID,shortest_dist
0,4,1044.86
1,5,932.50
2,6,1172.80
3,7,1015.92
4,8,1008.43
...,...,...
104074,135052,1115.61
104075,135056,1111.90
104076,135058,1117.25
104077,135059,1117.31


In [19]:
feature_df

,OBJECTID,FireType,FireYear,GISAcres,OverlapFlags
0,14299,Wildfire,1963,40992.458271,NaN
1,14300,Wildfire,1963,25757.090203,NaN
2,14301,Wildfire,1963,45527.210986,NaN
3,14302,Wildfire,1963,10395.010334,NaN
4,14303,Wildfire,1963,9983.605738,NaN
...,...,...,...,...,...
117573,135057,Prescribed Fire,2020,16.412148,"Caution, this Prescribed Fire in 2020 overlaps..."
117574,135058,Prescribed Fire,2020,7.050837,"Caution, this Prescribed Fire in 2020 overlaps..."
117575,135059,Prescribed Fire,2020,9.342668,"Caution, this Prescribed Fire in 2020 overlaps..."
117576,135060,Prescribed Fire,2020,0.996962,NaN


# Section 9: Data Checks

Lastly, we're checking and reporting some statistics about the fire data. We're looking at things like the most recent year, the earliest year, and the farthest fire from Salina. This helps us understand our data better.

In [24]:
filter_df = pd.merge(feature_df, miles_df, how = 'inner', left_on='OBJECTID', right_on='OBJECTID')
filter_df.to_csv('/Users/aviva/Desktop/MSDS/quarter_4/Human Centered Data Science/Part 1/GeoJSON Exports/intermediate_data/filter.csv', index=False)

In [25]:
max_year = filter_df['FireYear'].max()
min_year = filter_df['FireYear'].min()
max_dist = filter_df['shortest_dist'].max()

In [26]:
if max_year > 2023:
    print("Fires that occured after 2023 are present.")
else:
    print("Maximum fire year: {0}.".format(max_year))

if min_year < 1963:
    print("Fires that occured before 1963 are present.")
else:
    print("Minimum fire year: {0}.".format(min_year))

if max_dist > 1250:
    print("Fires that occured more than 1250 miles away from Salina are present.")
    print("The farthest fire is {0} miles from Salina.".format(max_dist))
    print("The closest fire is {0} miles from Salina.".format(filter_df['shortest_dist'].min()))

num_fires = len(filter_df)
print("There are {0} fires after 1963 within 1250 miles of Salina.".format(num_fires))

Maximum fire year is 2020.
Minimum fire year is 1963.
There are 91781 fires after 1963 within 1250 miles of Salina.


In [27]:
filter_df

,OBJECTID,FireType,FireYear,GISAcres,OverlapFlags,shortest_dist
0,14299,Wildfire,1963,40992.458271,NaN,1045.62
1,14300,Wildfire,1963,25757.090203,NaN,1074.55
2,14301,Wildfire,1963,45527.210986,NaN,1038.72
3,14302,Wildfire,1963,10395.010334,NaN,990.02
4,14303,Wildfire,1963,9983.605738,NaN,1034.55
...,...,...,...,...,...,...
91776,135052,Prescribed Fire,2020,60.879054,NaN,1115.61
91777,135056,Prescribed Fire,2020,14.545208,NaN,1111.90
91778,135058,Prescribed Fire,2020,7.050837,"Caution, this Prescribed Fire in 2020 overlaps...",1117.25
91779,135059,Prescribed Fire,2020,9.342668,"Caution, this Prescribed Fire in 2020 overlaps...",1117.31
